# 00 - MuJoCo Setup & Environment Test

## How to Run (Google Colab)

1. **Open in Colab**: Click "Open in Colab" badge or upload this notebook to Google Colab
2. **Run All Cells**: `Runtime` → `Run all` (or `Ctrl+F9`)
3. **Wait**: First cell installs dependencies (~1-2 minutes)

The notebook will:
- Clone the repository from GitHub
- Install MuJoCo and dependencies
- Test the soccer environment
- Verify everything works

---

**Note**: MuJoCo viewer is not available in Colab (no display), but all training, metrics, and plots work perfectly.


In [ ]:
# === Cell 1: Install dependencies and clone repo (Colab) ===
import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install dependencies
    print("Installing dependencies...")
    %pip install -q mujoco==3.1.6 gymnasium numpy matplotlib pandas
    
    # Clone repo if not already cloned
    REPO_URL = "https://github.com/zakik27/CSE337-RLSoccer.git"  # UPDATE THIS with your repo URL
    REPO_DIR = "/content/CSE337-RLSoccer"
    
    if not os.path.exists(REPO_DIR):
        print(f"Cloning repository...")
        !git clone {REPO_URL} {REPO_DIR}
    else:
        print(f"Repository already exists at {REPO_DIR}")
    
    # Change to repo directory
    os.chdir(REPO_DIR)
    PROJECT_ROOT = REPO_DIR
else:
    # Local setup (VS Code/Cursor)
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add to path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Define paths
XML_PATH = os.path.join(PROJECT_ROOT, "assets", "soccer_min.xml")
LOG_PATH = os.path.join(PROJECT_ROOT, "notebooks", "MUJOCO_LOG.TXT")

print(f"\n{'='*50}")
print(f"Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"Project Root: {PROJECT_ROOT}")
print(f"XML Path: {XML_PATH}")
print(f"{'='*50}")


In [ ]:
# === Cell 2: Test MuJoCo import and version ===
import mujoco
import numpy as np

print(f"MuJoCo version: {mujoco.__version__}")
print(f"NumPy version: {np.__version__}")

# Verify XML file exists
if os.path.exists(XML_PATH):
    print(f"✓ XML file found: {XML_PATH}")
else:
    print(f"✗ XML file NOT found: {XML_PATH}")


In [ ]:
# === Cell 3: Load environment and test ===
from src.soccer_env import SoccerKickEnv

# Create environment with fixed seed for reproducibility
SEED = 42
env = SoccerKickEnv(XML_PATH, seed=SEED)

print(f"Environment created successfully!")
print(f"Goal X position: {env.goal_x}")
print(f"Number of discrete actions: {len(env.actions)}")
print(f"Action space (yaw_rad, speed): {env.actions[:3]}... (showing first 3)")


In [ ]:
# === Cell 4: Reset and take one action ===
obs = env.reset(target_random=True)
print(f"Initial observation (relative to target): {obs}")

# Take a kick action (action index 12 = middle yaw, middle speed)
action_idx = 12
yaw_rad, speed = env.actions[action_idx]
print(f"\nTaking action {action_idx}: yaw={np.rad2deg(yaw_rad):.1f}°, speed={speed:.1f} m/s")

obs_next, reward, done, info = env.step(action_idx)

print(f"\nResult:")
print(f"  Reward: {reward:.3f}")
print(f"  Success (goal): {info['success']}")
print(f"  Cross position: {info['cross_pos']}")


In [ ]:
# === Cell 5: Log any MuJoCo warnings ===
import datetime

log_entry = f"""
--- MuJoCo Setup Test ---
Date: {datetime.datetime.now()}
MuJoCo version: {mujoco.__version__}
Environment loaded: OK
Action taken: OK
Result: reward={reward:.3f}, success={info['success']}
"""

with open(LOG_PATH, 'a') as f:
    f.write(log_entry)

print(f"Log appended to: {LOG_PATH}")


In [ ]:
# === Cell 6: Render a frame (works in Colab!) ===
import matplotlib.pyplot as plt

# Reset environment for a fresh view
env.reset(target_random=False)

# Render to an image array (headless - works in Colab)
renderer = mujoco.Renderer(env.model, height=480, width=640)
renderer.update_scene(env.data)
frame = renderer.render()

# Display the frame
plt.figure(figsize=(10, 7))
plt.imshow(frame)
plt.axis('off')
plt.title("Soccer Environment - MuJoCo Render")
plt.show()

print("✓ Headless rendering works! (No viewer window needed)")


In [ ]:
# === Cell 7: Summary ===
print("="*50)
print("MuJoCo Setup Test COMPLETE")
print("="*50)
print(f"✓ MuJoCo {mujoco.__version__} installed")
print(f"✓ Environment loads from assets/soccer_min.xml")
print(f"✓ Actions work ({len(env.actions)} discrete actions)")
print(f"✓ Reward/info returned correctly")
print(f"✓ Headless rendering works")
print(f"\nEnvironment: {'Google Colab' if IN_COLAB else 'Local'}")
print("\n→ Proceed to: 20_sarsa_kicker.ipynb for SARSA training")
